In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI

load_dotenv()
#openai.api_key = st.secrets.get("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))

#import cmdstanpy
#cmdstanpy.install_cmdstan()


# Definir las fechas globalmente
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

In [2]:
top_stocks = 'top_28_growth_stocks.csv'
df = pd.read_csv(top_stocks)

In [3]:
top_stocks_tickers = df['Ticker'].tolist()
top_stocks_names = df['name'].tolist()
print("top_stocks_tickers: ")
print(top_stocks_tickers)
print("top_stocks_names: ")
print(top_stocks_names)


top_stocks_tickers: 
['CWEN', 'CODQL', 'GNK', 'AVTR', 'FCN', 'MRTN', 'XYL', 'PDEX', 'ETR', 'SPRS', 'HURN', 'RSG', 'CWST', 'CTS', 'TRMB', 'ANSS', 'ADEA', 'NI', 'ACCO', 'ALE', 'CXT', 'HEI', 'MSA', 'RBC', 'MWA', 'ALB', 'LNG', 'CVLG']
top_stocks_names: 
['CLEARWAY ENERGY, INC.', 'CORONADO GLOBAL RESOURCES INC.', 'GENCO SHIPPING & TRADING LTD', 'AVANTOR, INC.', 'FTI CONSULTING, INC', 'MARTEN TRANSPORT LTD', 'XYLEM INC.', 'PRO DEX INC', 'ENTERGY CORP /DE/', 'SURGE COMPONENTS INC', 'HURON CONSULTING GROUP INC.', 'REPUBLIC SERVICES, INC.', 'CASELLA WASTE SYSTEMS INC', 'CTS CORP', 'TRIMBLE INC.', 'ANSYS INC', 'XPERI HOLDING CORP', 'NISOURCE INC.', 'ACCO BRANDS CORP', 'ALLETE INC', 'CRANE CO /DE/', 'HEICO CORP', 'MSA SAFETY INC', 'RBC BEARINGS INC', 'MUELLER WATER PRODUCTS, INC.', 'ALBEMARLE CORP', 'CHENIERE ENERGY INC', 'COVENANT LOGISTICS GROUP, INC.']


In [4]:
import yfinance as yf

# Comprobar si los tickers existen en yfinance
for ticker, name in zip(top_stocks_tickers, top_stocks_names):
    try:
        data = yf.Ticker(ticker).history(period="1d")
        if data.empty:
            print(f"Ticker NO válido o sin datos: {ticker} ({name})")
    except Exception as e:
        print(f"Error con {ticker} ({name}): {e}")

CWEN: No price data found, symbol may be delisted (period=1d)


Ticker NO válido o sin datos: CWEN (CLEARWAY ENERGY, INC.)


CODQL: No price data found, symbol may be delisted (period=1d)


Ticker NO válido o sin datos: CODQL (CORONADO GLOBAL RESOURCES INC.)


GNK: No price data found, symbol may be delisted (period=1d)


...

Ticker NO válido o sin datos: ALB (ALBEMARLE CORP)


LNG: No price data found, symbol may be delisted (period=1d)


Ticker NO válido o sin datos: LNG (CHENIERE ENERGY INC)


CVLG: No price data found, symbol may be delisted (period=1d)


Ticker NO válido o sin datos: CVLG (COVENANT LOGISTICS GROUP, INC.)


In [ ]:
# Descargar datos
@st.cache_data
def cargar_datos():
    
    # Descargar las empresas con mayor potencial tras aplicar el modelo de predicción
    top_stocks = 'top_28_growth_stocks.csv'
    df = pd.read_csv(top_stocks)
    # Creamos listas con los tickers y nombres de las empresas
    top_stocks_tickers = df['Ticker'].tolist()
    top_stocks_names = df['name'].tolist()
    print("top_stocks_tickers: ")
    print(top_stocks_tickers)
    print("top_stocks_names: ")
    print(top_stocks_names)
    # Comprobar si los tickers existen en yfinance
    for ticker, name in zip(top_stocks_tickers, top_stocks_names):
        try:
            data = yf.Ticker(ticker).history(period="1d")
            if data.empty:
                print(f"Ticker NO válido o sin datos: {ticker} ({name})")
        except Exception as e:
            print(f"Error con {ticker} ({name}): {e}")

    # Crear un diccionario con los tickers y nombres de las empresas
    symbols = dict(zip(top_stocks_names, top_stocks_tickers))

        # Puedes agregar más compañías aquí para tu MVP
    
    data = pd.DataFrame()
    for company, symbol in symbols.items():
        df = yf.download(symbol, start=start_date, end=end_date)['Close']
        if not df.empty:
            data[company] = df
        else:
            print(f"No se pudieron descargar los datos para {company} ({symbol}).")
    return data.dropna()

data = cargar_datos()

In [ ]:
# Rendimientos diarios
daily_returns = data.pct_change().dropna()
mean_daily_returns = daily_returns.mean()
cov_matrix = daily_returns.cov()

# Preguntas
st.title("📊 Recomendador de Cartera de Inversión")
st.write("Responde estas preguntas para determinar la cartera más adecuada a tu apetito de riesgo actual:")
st.write("**Esta simulación se basa en datos históricos y no garantiza rentabilidades futuras. Invierte con responsabilidad.*") # Disclaimer

# Widget con clave única
q1 = st.radio("Nivel de seguridad deseado:",
              ["Máxima seguridad", "Alta seguridad", "Equilibrado", "Alta rentabilidad", "Máxima rentabilidad"],
              index=None, key='q1')

q2 = st.radio("Diversificación deseada:",
              ["Muy conservadora", "Conservadora", "Equilibrada", "Agresiva", "Muy agresiva"],
              index=None, key='q2')

q3 = st.radio("Expectativa de rentabilidad:",
              ["Muy baja", "Moderada-baja", "Media", "Alta", "Muy alta"],
              index=None, key='q3')

q4 = st.radio("Nivel máximo de pérdida aceptable:",
              ["0%", "Hasta 5%", "Hasta 10%", "Hasta 20%", "Más de 20%"],
              index=None, key='q4')

q5 = st.radio("Horizonte temporal de la inversión:",
              ["< 1 año", "1-3 años", "3-5 años", "5-10 años", ">10 años"],
              index=None, key='q5')

# Botón para generar cartera (para evitar auto-ejecución)
if st.button("Generar cartera óptima"):

    # Verificar si todas las preguntas han sido respondidas
    if None in [q1, q2, q3, q4, q5]:
        st.warning("Por favor, responde todas las preguntas antes de continuar.")
    else:
        # Calcular puntuación
        preguntas = [q1, q2, q3, q4, q5]
        puntuaciones = [ ["Máxima seguridad", "Alta seguridad", "Equilibrado", "Alta rentabilidad", "Máxima rentabilidad"].index(q1)+1,
                         ["Muy conservadora", "Conservadora", "Equilibrada", "Agresiva", "Muy agresiva"].index(q2)+1,
                         ["Muy baja", "Moderada-baja", "Media", "Alta", "Muy alta"].index(q3)+1,
                         ["0%", "Hasta 5%", "Hasta 10%", "Hasta 20%", "Más de 20%"].index(q4)+1,
                         ["< 1 año", "1-3 años", "3-5 años", "5-10 años", ">10 años"].index(q5)+1 ]
        
        total = sum(puntuaciones)

        # Categorizar usuario
        if total <= 8:
            perfil = "Riesgo mínimo"
        elif total <= 12:
            perfil = "Riesgo bajo"
        elif total <= 17:
            perfil = "Riesgo medio"
        elif total <= 21:
            perfil = "Riesgo alto"
        else:
            perfil = "Rentabilidad máxima"

        st.subheader(f"Tu perfil actual: {perfil}")

        # Funciones objetivo
        def annualized_return(w):
            return np.sum(mean_daily_returns * w) * 252

        def annualized_volatility(w):
            return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w))) * np.sqrt(252)

        objetivos = {
            "Riesgo mínimo": lambda w: annualized_volatility(w),
            "Riesgo bajo": lambda w: 0.75*annualized_volatility(w)-0.25*annualized_return(w),
            "Riesgo medio": lambda w: 0.50*annualized_volatility(w)-0.50*annualized_return(w),
            "Riesgo alto": lambda w: 0.25*annualized_volatility(w)-0.75*annualized_return(w),
            "Rentabilidad máxima": lambda w: -annualized_return(w)
        }

        # Optimizar cartera
        def optimizar(objetivo):
            num_activos = len(mean_daily_returns)
            w0 = np.ones(num_activos) / num_activos
            bounds = [(0,1) for _ in range(num_activos)]
            constraints = {'type':'eq', 'fun': lambda w: np.sum(w)-1}
            result = minimize(objetivo, w0, bounds=bounds, constraints=constraints)
            return result.x

        # Cartera óptima
        pesos_optimos = optimizar(objetivos[perfil])

        # Mostrar resultados ordenados
        cartera = {data.columns[i]: round(pesos_optimos[i]*100, 2) for i in range(len(data.columns)) if pesos_optimos[i] > 0.001}

        # Ordenar la cartera por porcentaje de mayor a menor
        cartera_ordenada = dict(sorted(cartera.items(), key=lambda item: item[1], reverse=True))

        st.subheader("🎯 Composición óptima de tu cartera (ordenada de mayor a menor)")
        for empresa, peso in cartera_ordenada.items():
            st.write(f"- **{empresa}:** {peso}%")

        # Convertir divisa a EUR si es necesario

        # Mapeo de divisas por activo (según Yahoo Finance)
        divisas = {
            "AAPL": "USD", "MSFT": "USD", "AMZN": "USD", "GOOGL": "USD", "TSLA": "USD",
            "BRK-B": "USD", "JNJ": "USD", "XOM": "USD", "JPM": "USD", "V": "USD",
            "NESN.SW": "CHF", "ROG.SW": "CHF", "005930.KS": "KRW", "7203.T": "JPY", "6758.T": "JPY",
            "BABA": "HKD", "0700.HK": "HKD", "HSBA.L": "GBP", "BP.L": "GBP", "SHEL.L": "GBP",
            "ULVR.L": "GBP", "MC.PA": "EUR", "TTE.PA": "EUR", "SIE.DE": "EUR", "VOW3.DE": "EUR",
            "SAP.DE": "EUR", "SAN.MC": "EUR", "BBVA.MC": "EUR", "PBR": "BRL", "VALE": "BRL",
            "IBN": "INR", "RELIANCE.BO": "INR", "INFY": "INR", "2318.HK": "HKD",
            "0941.HK": "HKD", "RIO.L": "GBP", "BHP.AX": "AUD", "CBA.AX": "AUD",
            "CSL.AX": "AUD", "NOVO-B.CO": "DKK", "AZN.L": "GBP", "ADS.DE": "EUR",
            "HEIA.AS": "EUR", "PHIA.AS": "EUR", "ENEL.MI": "EUR", "RACE": "EUR", "2222.SR": "SAR",
            "TME": "USD", "META": "USD", "PG": "USD", "KO": "USD",
            "PEP": "USD", "MCD": "USD", "WMT": "USD", "COST": "USD", "INTC": "USD",
            "AMD": "USD", "NVDA": "USD", "QCOM": "USD", "AVGO": "USD", "TXN": "USD",
            "IBM": "USD", "ORCL": "USD", "CRM": "USD", "ADBE": "USD", "NFLX": "USD",
            "T": "USD", "VZ": "USD", "TMUS": "USD", "PFE": "USD", "MRNA": "USD",
            "MRK": "USD", "BMY": "USD", "AMGN": "USD", "GILD": "USD", "LLY": "USD",
            "CVX": "USD", "COP": "USD", "SLB": "USD", "HAL": "USD", "MRO": "USD",
            "LMT": "USD", "NOC": "USD", "RTX": "USD", "BA": "USD",
            "GE": "USD", "HON": "USD", "MMM": "USD", "CAT": "USD", "DE": "USD",
            "SBUX": "USD", "NKE": "USD", "LULU": "USD", "EL": "USD", "DPZ": "USD",
            "BKNG": "USD", "AXP": "USD", "MA": "USD", "PYPL": "USD"
        }



In [6]:
# Generar diccionario de divisas automáticamente
divisas = {}
for ticker in top_stocks_tickers:
    try:
        info = yf.Ticker(ticker).info
        currency = info.get('currency', 'USD')  # Por defecto USD si no se encuentra
        divisas[ticker] = currency
    except Exception as e:
        print(f"Error obteniendo divisa para {ticker}: {e}")
        divisas[ticker] = 'USD'  # Asume USD si hay error

print("Diccionario de divisas actualizado:")
print(divisas)

Diccionario de divisas actualizado:
{'CWEN': 'USD', 'CODQL': 'USD', 'GNK': 'USD', 'AVTR': 'USD', 'FCN': 'USD', 'MRTN': 'USD', 'XYL': 'USD', 'PDEX': 'USD', 'ETR': 'USD', 'SPRS': 'USD', 'HURN': 'USD', 'RSG': 'USD', 'CWST': 'USD', 'CTS': 'USD', 'TRMB': 'USD', 'ANSS': 'USD', 'ADEA': 'USD', 'NI': 'USD', 'ACCO': 'USD', 'ALE': 'USD', 'CXT': 'USD', 'HEI': 'USD', 'MSA': 'USD', 'RBC': 'USD', 'MWA': 'USD', 'ALB': 'USD', 'LNG': 'USD', 'CVLG': 'USD'}


In [ ]:
        # Descargar tipos de cambio necesarios
        st.write("🔄 **Descargando tipos de cambio para convertir a EUR...**")
        tipos_cambio = {}
        for divisa in set(divisas.values()):
            if divisa != "EUR":  # No necesitamos convertir EUR a EUR
                ticker = f"EUR{divisa}=X" if divisa != "USD" else "EURUSD=X"
                tipos_cambio[divisa] = yf.download(ticker, start=start_date, end=end_date)['Close'].fillna(method='ffill')

        # Convertir cada activo a EUR según su divisa
        st.write("💱 **Convirtiendo valores de los activos a EUR...**")
        data_eur = data.copy()
        for column in data.columns:
            divisa = divisas.get(column, "USD")  # Asumir USD por defecto si no se especifica
            if divisa != "EUR":  # Solo convertir si no está ya en EUR
                ticker = f"EUR{divisa}=X" if divisa != "USD" else "EURUSD=X"
                # Alinear los índices de los datos y los tipos de cambio
                tipo_cambio_alineado = tipos_cambio[divisa].reindex(data.index).fillna(method='ffill')
                # Convertir el precio del activo a EUR
                data_eur[column] = data[column] / tipo_cambio_alineado[ticker]

        # Continuar con el cálculo de la cartera
        st.write("📊 **Calculando el valor diario de la cartera propuesta...**")

        # Escalamos la cartera como si hubieras invertido 1.000 €
        inversion_inicial = 1000
        cartera_retornos = (data_eur * pesos_optimos).sum(axis=1)
        cartera_valores = cartera_retornos / cartera_retornos.iloc[0] * inversion_inicial